In [1]:
import torch
from torch import nn
from torchinfo import summary

from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm.auto import tqdm
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy  as np
import tracemalloc 

import json
import os
import glob
import pickle
from itertools import combinations
import gc
import time
import random

import warnings
warnings.filterwarnings("ignore")

/home/tran/miniconda3/envs/compgan/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TEST = False

In [3]:
DATA_ROOT = "../data/compgan_dataset/"

train_data_file_name_ = "train_data{}.npy"
train_label_file_name_ = "train_label{}.npy"
test_data_file_name_ = "test_data{}.npy"
test_label_file_name_ = "test_label{}.npy"

TRAIN_FOLDER_PATH = os.path.join(DATA_ROOT, "train")
TEST_FOLDER_PATH = os.path.join(DATA_ROOT, "test")
RESULT_FOLDER_PATH = os.path.join(DATA_ROOT, "results")
PREV_CHECKPOINT_FOLDER_PATH = os.path.join(DATA_ROOT, "prev_checkpoint")

assert os.path.isdir(TRAIN_FOLDER_PATH) and os.path.isdir(TEST_FOLDER_PATH)
os.makedirs(RESULT_FOLDER_PATH, exist_ok=True)

data_files = sorted(glob.glob(os.path.join(TRAIN_FOLDER_PATH, train_data_file_name_.format("*"))))
print(data_files)
print(len(data_files))

['../data/compgan_dataset/train/train_data0.npy', '../data/compgan_dataset/train/train_data1.npy', '../data/compgan_dataset/train/train_data10.npy', '../data/compgan_dataset/train/train_data11.npy', '../data/compgan_dataset/train/train_data12.npy', '../data/compgan_dataset/train/train_data13.npy', '../data/compgan_dataset/train/train_data14.npy', '../data/compgan_dataset/train/train_data15.npy', '../data/compgan_dataset/train/train_data2.npy', '../data/compgan_dataset/train/train_data3.npy', '../data/compgan_dataset/train/train_data4.npy', '../data/compgan_dataset/train/train_data5.npy', '../data/compgan_dataset/train/train_data6.npy', '../data/compgan_dataset/train/train_data7.npy', '../data/compgan_dataset/train/train_data8.npy', '../data/compgan_dataset/train/train_data9.npy']
16


In [4]:
USER_NUM = 16
SENSOR_NUM = 7
EACH_SENSOR_CHANNEL = 6
assert USER_NUM == len(data_files)
feature_num = SENSOR_NUM * EACH_SENSOR_CHANNEL

In [5]:
# important

label_list = ['歩行(平地)',
 '歩行(階段)',
 'ベッド上での起き上がり',
 'ベッド椅子間の乗り移り(立つ)',
 'ベッド椅子間の乗り移り(立たない)',
 '立ち座り',
 '座位保持・座位バランス',
 '立位保持・立位バランス',
 '関節可動域増大訓練(肩)',
 '関節可動域増大訓練(股関節)']

label_dict = dict(enumerate(label_list))

In [6]:
# important
eng_label_dict = dict(zip(
    label_list,
    ['Walking', 'Upstair', 'Bed_Standup', 'Change_Bed', 'Change_Bed_Standup', 'Sit_Down', 'Sit', 'Stand', 'Shoulder_Exercise', 'Hip_Exercise']
))

eng_label_list = [eng_label_dict[i] for i in label_list]

In [7]:
# Update after 2023/11/11 houkoku
class CustomTrainDataset(Dataset):
    TRAIN_MODE = "train"
    TEST_MODE = "test"
    
    def __init__(self, mode, feature_data, label_data, missing_sensor_numbers=0):
        self.mode = mode
        assert mode in [self.TRAIN_MODE, self.TEST_MODE]
        
        self.features = feature_data
        self.label = label_data
        assert len(self.features) == len(self.label), "features len is not equal to label len"
        self.missing_sensor_numbers = missing_sensor_numbers

        self.missing_index_list = []
        for missing_count in range(missing_sensor_numbers + 1):
            for missing_index in combinations(range(SENSOR_NUM), missing_count):
                self.missing_index_list.append(missing_index)

    def transform(self, one_feature, missing_sensor_id_list):
        # Make one sensor data become 0
        one_feature_cp = one_feature.copy()
        
        for missing_sensor_id in missing_sensor_id_list:
            one_feature_cp[:, missing_sensor_id*6:(missing_sensor_id+1)*6] = 0
        return one_feature_cp
        
    def __len__(self):

        # take all available missing pattern * data number
        return len(self.features) * len(self.missing_index_list)
    
    def __getitem__(self, idx):
        # take all available missing pattern
        missing_sensor_id_list = self.missing_index_list[ idx // len(self.features) ]
        x = self.transform(self.features[ idx % len(self.features) ], missing_sensor_id_list)
        label = self.label[idx % len(self.features)]
        return x, int(label)


In [8]:
class DataModule(pl.LightningDataModule):
    STANDARDIZE = False
    
    def __init__(self, test_user, missing_sensor_numbers, batch_size=2048):
        super().__init__()
        self.test_user = test_user
        self.missing_sensor_numbers = missing_sensor_numbers
        self.batch_size = batch_size

    def load_data(self, mode):
        if mode == "train":
            folder_path = TRAIN_FOLDER_PATH
            data_file_name = train_data_file_name_
            label_file_name = train_label_file_name_

            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            train_val_data = train_val_data.reshape(l, s * d, w).transpose(0,2,1)
            
        elif mode == "test":
            folder_path = TEST_FOLDER_PATH
            data_file_name = test_data_file_name_
            label_file_name = test_label_file_name_
    
            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            train_val_data = train_val_data.reshape(l, s * d, w).transpose(0,2,1)
    
        return train_val_data, train_val_label

    def setup(self, stage: str):
        # Assign Train/val split(s) for use in Dataloaders
        
        if stage == "validate" or stage == "fit":
            self.train_data, self.train_label = self.load_data("train")
            self.train_dataset = CustomTrainDataset(
                CustomTrainDataset.TRAIN_MODE, self.train_data, self.train_label, missing_sensor_numbers=self.missing_sensor_numbers)
            
            self.val_data, self.val_label = self.load_data("test")
            self.val_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.val_data, self.val_label, missing_sensor_numbers=self.missing_sensor_numbers)

        elif stage == "test" or stage == "predict":
            self.test_data, self.test_label = self.load_data("test")
            self.test_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.test_data, self.test_label, missing_sensor_numbers=self.missing_sensor_numbers)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=4, shuffle=True, pin_memory=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def teardown(self, stage):
        print("teardown")
        if stage == "validate" or stage == "fit":
            del self.train_data, self.train_label
            del self.val_data, self.val_label
            del self.train_dataset
            del self.val_dataset
            
        elif stage == "test" or stage == "predict":
            del self.test_data, self.test_label
            del self.test_dataset
        gc.collect()


In [9]:
if TEST:
    data_module = DataModule(test_user=0, missing_sensor_numbers=1)
    data_module.setup("fit")
    print(data_module.train_data.shape)
    print(data_module.val_data.shape)
    # print(data_module.test_data.shape)

In [10]:
if TEST:
    data = data_module.val_dataset[1273 * 2 + 1]
    print(data[1], data[0][0])
    print(len(data_module.val_dataset))

In [11]:
if TEST:
    del data_module, data
    gc.collect()

## MODEL DEFINITION

In [12]:

class LSTMModel(pl.LightningModule):
    def __init__(self, hidden_size=128, input_size=42, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, 256, input_size)
        
        self.rnn = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=3,
                          batch_first=True)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
    def forward(self, x):
        activation, _ = self.rnn(x)
        
        b, _, _ = activation.size()
        lstm_output = activation[:,-1,:].view(b,-1)
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [13]:
class LSTMBiModel(pl.LightningModule):
    def __init__(self, hidden_size=128, input_size=42, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, 256, input_size)
        
        self.rnn = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=3,
                          batch_first=True,
                          bidirectional=True)

        double_hidden_size = hidden_size * 2
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=double_hidden_size, out_features=double_hidden_size),
            nn.BatchNorm1d(num_features=double_hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * double_hidden_size, out_features=output_size)
        
    def forward(self, x):
        activation, (h, _) = self.rnn(x)
        b, _, _ = activation.size()
        
        lstm_output = h[-2:].permute(1,0,2).reshape(b, -1)
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [14]:
class LSTMAttentionModel(pl.LightningModule):
    def __init__(self, hidden_size=128, input_size=42, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, 256, input_size)
        
        self.rnn1 = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention1 = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            batch_first=True
        )

        self.rnn2 = nn.LSTM(input_size=hidden_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention2 = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            batch_first=True
        )

        self.rnn3 = nn.LSTM(input_size=hidden_size, 
                  hidden_size=hidden_size,
                  num_layers=1,
                  batch_first=True)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
    def forward(self, x):
        activation, _ = self.rnn1(x)
        activation, _ = self.attention1(activation, activation, activation)
        activation, _ = self.rnn2(activation)
        activation, _ = self.attention2(activation, activation, activation)
        activation, (h, _) = self.rnn3(activation)

        b, _, _ = activation.size()
        
        lstm_output = activation[:,-1,:].view(b,-1)
        
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [15]:

class CNNLSTMModel(pl.LightningModule):
    def __init__(self, hidden_size=128, sequence_length=256, input_size=42, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, sequence_length, input_size)
        
        self.cnn = nn.Conv1d(sequence_length, cnn_filter_size, kernel_size=5, padding="same")
        
        self.rnn = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=3,
                          batch_first=True)
 
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
    def forward(self, x):
        output = self.cnn(x)
        activation, _ = self.rnn(output)
        
        b, _, _ = activation.size()
        lstm_output = activation[:,-1,:].view(b,-1)
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [16]:
class CNNLSTMAttentionModel(pl.LightningModule):
    def __init__(self, hidden_size=128, sequence_length=256, input_size=42, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, sequence_length, input_size)
        
        self.cnn = nn.Conv1d(sequence_length, cnn_filter_size, kernel_size=5, padding="same")
        
        self.rnn1 = nn.LSTM(input_size=input_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention1 = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            batch_first=True
        )
        
        self.rnn2 = nn.LSTM(input_size=hidden_size, 
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
        self.attention2 = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            batch_first=True
        )

        self.rnn3 = nn.LSTM(input_size=hidden_size, 
                  hidden_size=hidden_size,
                  num_layers=1,
                  batch_first=True)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
    def forward(self, x):
        output = self.cnn(x)
        activation, _ = self.rnn1(output)
        activation, _ = self.attention1(activation, activation, activation)
        activation, _ = self.rnn2(activation)
        activation, _ = self.attention2(activation, activation, activation)
        activation, _ = self.rnn3(activation)

        b, _, _ = activation.size()
        
        lstm_output = activation[:,-1,:].view(b,-1)
        
        seq_1_output = self.seq_1(lstm_output)
        seq_2_output = self.seq_2(lstm_output)
        
        output = torch.concat([lstm_output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [17]:
class CNNModel(pl.LightningModule):
    def __init__(self, hidden_size=128, sequence_length=256, input_size=42, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, sequence_length, input_size)

        self.cnn1 = nn.Sequential(
            nn.Conv1d(sequence_length, cnn_filter_size, kernel_size=5, padding="same"),
            nn.BatchNorm1d(num_features=cnn_filter_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(cnn_filter_size, hidden_size, kernel_size=5, padding="same"),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )

        self.gap = nn.AdaptiveAvgPool1d(1)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
    def forward(self, x):
        output = self.cnn1(x)
        output = self.cnn2(output)
        b, _, _ = output.shape
        
        output = self.gap(output).view(b, -1)

        seq_1_output = self.seq_1(output)
        seq_2_output = self.seq_2(output)
        
        output = torch.concat([output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [18]:
class CNNAttentionModel(pl.LightningModule):
    def __init__(self, hidden_size=128, sequence_length=256, input_size=42, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(1024, sequence_length, input_size)

        self.cnn1 = nn.Sequential(
            nn.Conv1d(sequence_length, cnn_filter_size, kernel_size=5, padding="same"),
            nn.BatchNorm1d(num_features=cnn_filter_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )

        self.attention1 =  nn.MultiheadAttention(
            embed_dim=input_size,
            num_heads=6,
            batch_first=True
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(cnn_filter_size, hidden_size, kernel_size=5, padding="same"),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )

        self.attention2 =  nn.MultiheadAttention(
            embed_dim=input_size,
            num_heads=6,
            batch_first=True
        )
        
        self.gap = nn.AdaptiveAvgPool1d(1)
        
        self.seq_1 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.seq_2 = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.BatchNorm1d(num_features=hidden_size),
            nn.Dropout1d(p=0.2),
            nn.ReLU(),
        )
        
        self.classifier = nn.Linear(in_features=3 * hidden_size, out_features=output_size)
        
    def forward(self, x):
        output = self.cnn1(x)
        output, _ = self.attention1(output, output, output)
        output = self.cnn2(output)
        output, _ = self.attention2(output, output, output)
        b, _, _ = output.shape
        
        output = self.gap(output).view(b, -1)

        seq_1_output = self.seq_1(output)
        seq_2_output = self.seq_2(output)
        
        output = torch.concat([output, seq_1_output, seq_2_output], dim=1)
        output = self.classifier(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [19]:
# train_model_class = [LSTMModel, LSTMAttentionModel, CNNLSTMModel, CNNLSTMAttentionModel, CNNAttentionModel, CNNModel]
# train_model_class = [CNNModel, CNNAttentionModel]

# user 0: tran
# user 1: hachix
train_model_class = [CNNLSTMAttentionModel, CNNAttentionModel, LSTMAttentionModel, CNNModel, LSTMModel, CNNLSTMModel]
# train_model_class = [CNNLSTMAttentionModel]


In [ ]:
# only for test

def test():
    patience = 20
    missing_sensor_numbers = 4 # no missing sensor
    user = 2 # use user2 to test
    batch_size_dict = {LSTMAttentionModel:512}
    for model_class in train_model_class:
        data_module = DataModule(test_user=user, 
                                 missing_sensor_numbers=missing_sensor_numbers, 
                                 batch_size=batch_size_dict.get(model_class, 1024))
        
        model = model_class(input_size=42, output_size=10)
        
        model_name = model.__class__.__name__
        print("Running for model", model_name)
        print("summary(model)", summary(model))

        tb_logger = TensorBoardLogger(f"./loggers/{model_name}")
        
        trainer = pl.Trainer(
            logger=tb_logger,
            callbacks=[EarlyStopping(monitor="val_loss", patience=patience, mode="min")],
            fast_dev_run = True
        )
        trainer.fit(model, data_module)
        trainer.test(model, data_module)
if TEST:
    test()

In [23]:
patience = 200
# missing_sensor_numbers = 0
# all_test_pred = {}
batch_size_dict = {LSTMAttentionModel:2048, LSTMModel:2048}
for model_class in train_model_class: 
    
    model = model_class(input_size=42, output_size=10)
    model_name = model.__class__.__name__
    print("Running for model", model_name)
    
    for missing_sensor_numbers in [6]: ## Changed for 1 missing sensor
        for user in [0]: ## Changed for user 2 only
            # Load from exist previous training cpkt for continuous learning & testing
            # previous_checkpoint_file = os.path.join(PREV_CHECKPOINT_FOLDER_PATH, f"{model_name}_{missing_sensor_numbers}missing_user{user}.ckpt")
            # previous_checkpoint_file = "/home/tran/acttivity_recognition/data/compgan_dataset/results/logger86_6missing/CNNLSTMAttentionModel/user0/bk/checkpoints/epoch=259-step=16640.ckpt"
            # if os.path.isfile(previous_checkpoint_file):
            #     model = model_class.load_from_checkpoint(previous_checkpoint_file)
            #     print("loaded", model_name, missing_sensor_numbers, user)
                
            start_timer = time.perf_counter()
            print(f"\n*************training on User{user}*************")
            
            data_module = DataModule(test_user=user, 
                                     missing_sensor_numbers=missing_sensor_numbers,
                                     batch_size=batch_size_dict.get(model_class, 4096))

            save_dir = os.path.join(RESULT_FOLDER_PATH, f"./logger86_{missing_sensor_numbers}missing/{model_name}")
            save_dir_name = f"user{user}"
            
            tb_logger = TensorBoardLogger(save_dir=save_dir, name=save_dir_name)
            
            # checkpoint_callback = ModelCheckpoint(
            #     dirpath=None,
            #     save_top_k=1,  # get the 2 minimum val loss checkpoint
            #     monitor="val_loss",
            #     mode="min",
            #     filename="best-{epoch:02d}-{step:02d}-{val_loss:.2f}"
            # )
            
            trainer = pl.Trainer(
                logger=tb_logger,
                callbacks=[
                    EarlyStopping(monitor="val_loss", patience=patience, mode="min"),
                    LearningRateMonitor("epoch"),
                    # checkpoint_callback,
                ],
                precision="16-mixed",
                # precision=16,
                accumulate_grad_batches=10,
                log_every_n_steps=200,
                check_val_every_n_epoch=5,
                accelerator="auto",
                devices=1,
                max_epochs=100000,
            )
        
            trainer.fit(model, data_module)
            trainer.test(model, data_module)
    
            end_timer = time.perf_counter()
            exec_time = end_timer - start_timer

            trainer_test_dict = trainer.logged_metrics
            for key in trainer_test_dict.keys():
                trainer_test_dict[key] = trainer_test_dict[key].item()
            trainer_test_dict["exec_time"] = int(exec_time)
            
            with open(os.path.join(trainer.log_dir, f"result.json"), "w") as f:
                json.dump(trainer_test_dict, f)


Running for model LSTMModel
loaded LSTMModel 0 0
loaded LSTMModel 0 1
loaded LSTMModel 0 2
loaded LSTMModel 0 3
loaded LSTMModel 0 4
loaded LSTMModel 0 5
loaded LSTMModel 0 6
loaded LSTMModel 0 7
loaded LSTMModel 0 8
loaded LSTMModel 0 9
loaded LSTMModel 0 10
loaded LSTMModel 0 11
loaded LSTMModel 0 12
loaded LSTMModel 0 13
loaded LSTMModel 0 14
loaded LSTMModel 0 15
loaded LSTMModel 1 0
loaded LSTMModel 1 1
loaded LSTMModel 1 2
loaded LSTMModel 1 3
loaded LSTMModel 1 4
loaded LSTMModel 1 5
loaded LSTMModel 1 6
loaded LSTMModel 1 7
loaded LSTMModel 1 8
loaded LSTMModel 1 9
loaded LSTMModel 1 10
loaded LSTMModel 1 11
loaded LSTMModel 1 12
loaded LSTMModel 1 13
loaded LSTMModel 1 14
loaded LSTMModel 1 15
Running for model LSTMAttentionModel
loaded LSTMAttentionModel 0 0
loaded LSTMAttentionModel 0 1
loaded LSTMAttentionModel 0 2
loaded LSTMAttentionModel 0 3
loaded LSTMAttentionModel 0 4
loaded LSTMAttentionModel 0 5
loaded LSTMAttentionModel 0 6
loaded LSTMAttentionModel 0 7
loaded LSTM

## Predict full length

In [20]:
# Update after 2023/11/11 houkoku
class CustomTestDataset(Dataset):
    
    def __init__(self, feature_data, label_data, missing_sensor_numbers=0):
        self.features = feature_data
        self.label = label_data
        assert len(self.features) == len(self.label), "features len is not equal to label len"
        self.missing_sensor_numbers = missing_sensor_numbers

        self.missing_index_list = []
        for missing_index in combinations(range(SENSOR_NUM), missing_sensor_numbers):
            self.missing_index_list.append(missing_index)

    def transform(self, one_feature, missing_sensor_id_list):
        # Make one sensor data become 0
        one_feature_cp = one_feature.copy()
        
        for missing_sensor_id in missing_sensor_id_list:
            one_feature_cp[:, missing_sensor_id*6:(missing_sensor_id+1)*6] = 0
        return one_feature_cp
        
    def __len__(self):

        # take all available missing pattern * data number
        return len(self.features) * len(self.missing_index_list)
    
    def __getitem__(self, idx):
        # take all available missing pattern
        missing_sensor_id_list = self.missing_index_list[ idx // len(self.features) ]
        x = self.transform(self.features[ idx % len(self.features) ], missing_sensor_id_list)
        label = self.label[idx % len(self.features)]
        return x, int(label)
    
class DataModule(pl.LightningDataModule):
    STANDARDIZE = False
    
    def __init__(self, test_user, missing_sensor_numbers, batch_size=2048):
        super().__init__()
        self.test_user = test_user
        self.missing_sensor_numbers = missing_sensor_numbers
        self.batch_size = batch_size

    def load_data(self, mode):
        if mode == "train":
            folder_path = TRAIN_FOLDER_PATH
            data_file_name = train_data_file_name_
            label_file_name = train_label_file_name_

            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            train_val_data = train_val_data.reshape(l, s * d, w).transpose(0,2,1)
            
        elif mode == "test":
            folder_path = TEST_FOLDER_PATH
            data_file_name = test_data_file_name_
            label_file_name = test_label_file_name_
    
            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            train_val_data = train_val_data.reshape(l, s * d, w).transpose(0,2,1)
    
        return train_val_data, train_val_label

    def setup(self, stage: str):
        # Assign Train/val split(s) for use in Dataloaders
        
        if stage == "validate" or stage == "fit":
            train_val_data, train_val_label = self.load_data("train")
            self.train_data, self.val_data, self.train_label, self.val_label = train_test_split(
                train_val_data, train_val_label, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

            self.train_dataset = CustomTestDataset(
                self.train_data, self.train_label, missing_sensor_numbers=self.missing_sensor_numbers)
            self.val_dataset = CustomTestDataset(
                self.val_data, self.val_label, missing_sensor_numbers=self.missing_sensor_numbers)

        elif stage == "test" or stage == "predict":
            train_val_data, train_val_label = self.load_data("test")
            self.test_data = train_val_data
            self.test_label = train_val_label

            self.test_dataset = CustomTestDataset(
                self.test_data, self.test_label, missing_sensor_numbers=self.missing_sensor_numbers)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=4, shuffle=True, pin_memory=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def teardown(self, stage):
        print("teardown")
        if stage == "validate" or stage == "fit":
            del self.train_data, self.train_label
            del self.val_data, self.val_label
            del self.train_dataset
            del self.val_dataset
            
        elif stage == "test" or stage == "predict":
            del self.test_data, self.test_label
            del self.test_dataset
        gc.collect()



In [24]:
all_test_pred = {}

for missing_sensor_numbers in range(SENSOR_NUM): ## Changed for 1 missing sensor
    all_test_pred[missing_sensor_numbers] = {}
    
    for model_class in train_model_class: 
        all_user_y_true = []
        all_user_y_pred = []
        
        for user in range(1): ## Changed for user 2 only
            model_name = model_class.__name__
            checkpoint_path = os.path.join(RESULT_FOLDER_PATH, f"logger86_6missing", f"{model_name}", f"user{user}", "version_0", "checkpoints", "*.ckpt")
            checkpoint_glob = glob.glob(checkpoint_path)
            assert len(checkpoint_glob) == 1, checkpoint_path
            checkpoint_file = checkpoint_glob[0]

            assert os.path.exists(checkpoint_file), checkpoint_file
            model = model_class.load_from_checkpoint(checkpoint_file)

            data_module = DataModule(test_user=user, missing_sensor_numbers=missing_sensor_numbers, batch_size=512)
            data_module.setup("test")
            test_loader = data_module.test_dataloader()
            
            tb_logger = TensorBoardLogger(os.path.join(RESULT_FOLDER_PATH, f"logger86_6missing", f"{model_name}", f"user{user}", "version_0", f"result_test"))
            trainer = pl.Trainer(logger=tb_logger)

            pred = trainer.predict(model, test_loader)

            y_true_list = []
            for batch_idx, test_data in enumerate(test_loader):
                _, y_true = test_data
                y_true_list.append(y_true)

            all_user_y_true.append(torch.cat(y_true_list))
            all_user_y_pred.append(torch.cat(pred))
        
        print("model_name", model_name)
        all_test_cpu = list(map(lambda x: x.cpu().item(), torch.cat(all_user_y_true)))
        all_pred_cpu = list(map(lambda x: x.cpu().item(), torch.cat(all_user_y_pred)))
        
        all_test_pred[missing_sensor_numbers][model_name] = (all_test_cpu, all_pred_cpu)
        
        print("accuracy", accuracy_score(all_test_cpu, all_pred_cpu))

Lightning automatically upgraded your loaded checkpoint from v1.6.1 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../data/compgan_dataset/results/logger91_6missing/CNNLSTMAttentionModel/user0/version_0/checkpoints/val_loss_min-epoch=419-val_loss=0.24-val_acc=0.00.ckpt`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 96.01it/s] 
model_name CNNLSTMAttentionModel
accuracy 0.48598130841121495


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 97.26it/s] 
model_name CNNAttentionModel
accuracy 0.705607476635514


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

model_name LSTMAttentionModel
accuracy 0.6361815754339119
Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 336.32it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model_name CNNModel
accuracy 0.4459279038718291
Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 28.20it/s] 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model_name LSTMModel
accuracy 0.5694259012016022
Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 78.31it/s] 
model_name CNNLSTMModel
accuracy 0.44659546061415223


In [26]:
df = pd.DataFrame(index=[f"missing_{x}" for x in range(SENSOR_NUM)])

for model_class in train_model_class: 
    acc_list = []
    for missing_sensor_numbers in range(7): ## Changed for 1 missing sensor
        for user in range(1):
            all_test_cpu, all_pred_cpu = all_test_pred[missing_sensor_numbers][model_class.__name__]
            acc = f1_score(all_test_cpu, all_pred_cpu, average="macro")
            acc_list.append(acc)
            
    df[model_class.__name__] = acc_list
df
df.to_csv("model_result_f1_score.csv")
        

In [23]:
df = pd.DataFrame(index=[f"missing_{x}" for x in range(SENSOR_NUM)])

for model_class in train_model_class: 
    acc_list = []
    for missing_sensor_numbers in range(7): ## Changed for 1 missing sensor
        for user in range(1):
            all_test_cpu, all_pred_cpu = all_test_pred[missing_sensor_numbers][model_class.__name__]
            acc = accuracy_score(all_test_cpu, all_pred_cpu)
            acc_list.append(acc)
            
    df[model_class.__name__] = acc_list
df
        

,CNNLSTMAttentionModel,CNNAttentionModel,LSTMAttentionModel,CNNModel,LSTMModel,CNNLSTMModel
missing_0,0.359813,0.705607,0.864486,0.878505,0.757009,0.677570
missing_1,0.434579,0.658211,0.844459,0.857143,0.686916,0.658879
missing_2,0.443703,0.634179,0.816199,0.812194,0.647753,0.629951
missing_3,0.449132,0.609613,0.765421,0.750467,0.642056,0.587450
missing_4,0.450868,0.596128,0.713351,0.677570,0.645661,0.568625
missing_5,0.431019,0.564085,0.680908,0.586782,0.639519,0.514909
missing_6,0.347130,0.510681,0.636182,0.445928,0.569426,0.446595
